# An Introduction to Dask

Dask is a flexible parallel computing library for Python. It allows dynamic scheduling of tasks for optimising computations and includes collections (extensions of arrays, lists and dataframes) for use with parallel data processing.

## Terminology

### General

#### Thread
A thread is the simplest unit of computation.

#### Process
A process is an isolated computation which consists of one or more threads. The threads within a process can be executed concurrently (simultaneously) and have access to the same resources within the process (memory, executable code and variable values). Processes cannot share these resources with other processes. Each process has its own individual address space, while the threads within that process share their address space.

#### Graph
A way to represent 'things' and their relationships, where a node (circle) is a thing and an edge (line) between nodes is their relationship; e.g. a social network where nodes are people and edges are their friendships.


### Dask

#### Dask architecture
Below is the basic structure of how the components of dask interact:

![](../images/dask_architecture.png)

Dask uses a pool of workers to process tasks specified by the scheduler. The scheduler determines the tasks to be done by intelligently traversing the task graph. The task graph is produced dynamically and automatically passed to the scheduler when using dask functionality in your code. In the case of dask distributed, the task graph is submitted to the scheduler by the client.

#### Scheduler
One of Dask’s key benefits is its ability to efficiently schedule tasks to optimize computations. Everything Dask does is built on top of “schedulers”. These schedulers take the order of work established by a task graph and find the optimal way to break down and carry out the tasks.

Dask has four types of schedulers:
- Synchronous: Single thread (good for debugging).
        dask.get
- Threaded: Utilises a thread pool.
        dask.threaded.get
- Multiprocessing: Utilises a process pool.
        dask.multiprocessing.get
- Distributed: Utilises a cluster of distributed machines.
        distributed.Client.get

#### Worker
A worker receives tasks to process from the scheduler and returns output to the scheduler when that processing is finished. A worker can be a thread, a process or a whole machine, depending on the scheduler used.

#### Client
A client provides the primary point of access to a distributed scheduler and its associated workers. When using a threaded or multiprocessing scheduler interation with the scheduler is handled by the collection or delayed object.


#### Task Graph
A graph of tasks (nodes) and the data which is required to pass between them (edges).
In the context of dask:
- Task = circle
- Data = box
- Direction of flow = arrow

In [8]:
def add(a, b):
    return a + b

x = 1
y = 2
z = add(x, y)
w = sum([x, y, z])

![](../images/task_graph_def_img_1.png)

Viewing the relationship between tasks and data can reveal better\* ways to order the tasks to achieve the same computation.

\* Better can mean in less time or using less memory or both!

#### Dask Graph
Dask stores task graphs in a Python dictionary which maps keys to computations:

In [ ]:
dsk = {'x': 1,
       'y': 2,
       'z': (add, 'x', 'y'),
       'w': (sum, ['x', 'y', 'z'])
      }

Dask is different from other parallelising libraries in that it uses ordinary Python structures to represent task graphs instead of a specialised API:

- {Dicts}
- (Tuples)
- Functions()
- Values


## Exercise: Terminology

Now it is time to test what you have learned! Answer the following by matching the definitions with the terms.

Q: What is a scheduler? (Answer in the cell below.)

Q: What is a thread?

Q: What is a task graph?

Q: What is a process?

Q: What is a client?